In [0]:
pip install tensorflow==1.3

In [0]:
#@title Default title text
pip install awscli

In [0]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import shutil
shutil.unpack_archive("/content/drive/My Drive/solutions.zip", "/content/Input")

In [0]:
#@title data_utils
"""
Utilities for data processing.
"""

import tensorflow as tf
import numpy as np
import os

"""
File formatting note.
Data should be preprocessed as a sequence of comma-seperated ints with
sequences  /n seperated
"""

# Lookup tables
aa_to_int = {
    'M':1,
    'R':2,
    'H':3,
    'K':4,
    'D':5,
    'E':6,
    'S':7,
    'T':8,
    'N':9,
    'Q':10,
    'C':11,
    'U':12,
    'G':13,
    'P':14,
    'A':15,
    'V':16,
    'I':17,
    'F':18,
    'Y':19,
    'W':20,
    'L':21,
    'O':22, #Pyrrolysine
    'X':23, # Unknown
    'Z':23, # Glutamic acid or GLutamine
    'B':23, # Asparagine or aspartic acid
    'J':23, # Leucine or isoleucine
    'start':24,
    'stop':25,
}

int_to_aa = {value:key for key, value in aa_to_int.items()}

def get_aa_to_int():
    """
    Get the lookup table (for easy import)
    """
    return aa_to_int

def get_int_to_aa():
    """
    Get the lookup table (for easy import)
    """
    return int_to_aa

# Helper functions

def aa_seq_to_int(s):
    """
    Return the int sequence as a list for a given string of amino acids
    """
    return [24] + [aa_to_int[a] for a in s] + [25]

def int_seq_to_aa(s):
    """
    Return the int sequence as a list for a given string of amino acids
    """
    return "".join([int_to_aa[i] for i in s])

def tf_str_len(s):
    """
    Returns length of tf.string s
    """
    return tf.size(tf.string_split([s],""))

def tf_rank1_tensor_len(t):
    """
    Returns the length of a rank 1 tensor t as rank 0 int32
    """
    l = tf.reduce_sum(tf.sign(tf.abs(t)), 0)
    return tf.cast(l, tf.int32)


def tf_seq_to_tensor(s):
    """
    Input a tf.string of comma seperated integers.
    Returns Rank 1 tensor the length of the input sequence of type int32
    """
    return tf.string_to_number(
        tf.sparse_tensor_to_dense(tf.string_split([s],","), default_value='0'), out_type=tf.int32
    )[0]

def smart_length(length, bucket_bounds=tf.constant([128, 256])):
    """
    Hash the given length into the windows given by bucket bounds. 
    """
    # num_buckets = tf_len(bucket_bounds) + tf.constant(1)
    # Subtract length so that smaller bins are negative, then take sign
    # Eg: len is 129, sign = [-1,1]    
    signed = tf.sign(bucket_bounds - length)
    
    # Now make 1 everywhere that length is greater than bound, else 0
    greater = tf.sign(tf.abs(signed - tf.constant(1)))
    
    # Now simply sum to count the number of bounds smaller than length
    key = tf.cast(tf.reduce_sum(greater), tf.int64)
    
    # This will be between 0 and len(bucket_bounds)
    return key

def pad_batch(ds, batch_size, padding=None, padded_shapes=([None])):
    """
    Helper for bucket batch pad- pads with zeros
    """
    return ds.padded_batch(batch_size, 
                           padded_shapes=padded_shapes,
                           padding_values=padding
                          )

def aas_to_int_seq(aa_seq):
    int_seq = ""
    for aa in aa_seq:
        int_seq += str(aa_to_int[aa]) + ","
    return str(aa_to_int['start']) + "," + int_seq + str(aa_to_int['stop'])

# Preprocessing in python
def fasta_to_input_format(source, destination):
    # I don't know exactly how to do this in tf, so resorting to python.
    # Should go line by line so everything is not loaded into memory
    
    sourcefile = os.path.join(source)
    destination = os.path.join(destiation)
    with open(sourcefile, 'r') as f:
        with open(destination, 'w') as dest:
            seq = ""
            for line in f:
                if line[0] == '>' and not seq == "":
                    dest.write(aas_to_int_seq(seq) + '\n')
                    seq = ""
                elif not line[0] == '>':
                    seq += line.replace("\n","")

# Real data pipelines

def bucketbatchpad(
        batch_size=256,
        path_to_data=os.path.join("./data/SwissProt/sprot_ints.fasta"), # Preprocessed- see note
        compressed="", # See tf.contrib.data.TextLineDataset init args
        bounds=[128,256], # Default buckets of < 128, 128><256, >256
        # Unclear exactly what this does, should proly equal batchsize
        window_size=256, # NOT a tensor 
        padding=None, # Use default padding of zero, otherwise see Dataset docs
        shuffle_buffer=None, # None or the size of the buffer to shuffle with
        pad_shape=([None]),
        repeat=1,
        filt=None

):
    """
    Streams data from path_to_data that is correctly preprocessed.
    Divides into buckets given by bounds and pads to full length.
    Returns a dataset which will return a padded batch of batchsize
    with iteration.
    """
    batch_size=tf.constant(batch_size, tf.int64)
    bounds=tf.constant(bounds)
    window_size=tf.constant(window_size, tf.int64)
    
    path_to_data = os.path.join(path_to_data)
    # Parse strings to tensors
    dataset = tf.contrib.data.TextLineDataset(path_to_data).map(tf_seq_to_tensor)
    if filt is not None:
        dataset = dataset.filter(filt)

    if shuffle_buffer:
        # Stream elements uniformly randomly from a buffer
        dataset = dataset.shuffle(buffer_size=shuffle_buffer)
    # Apply a repeat. Because this is after the shuffle, all elements of the dataset should be seen before repeat.
    # See https://stackoverflow.com/questions/44132307/tf-contrib-data-dataset-repeat-with-shuffle-notice-epoch-end-mixed-epochs
    dataset = dataset.repeat(count=repeat)
    # Apply grouping to bucket and pad
    grouped_dataset = dataset.group_by_window(
        key_func=lambda seq: smart_length(tf_rank1_tensor_len(seq), bucket_bounds=bounds), # choose a bucket
        reduce_func=lambda key, ds: pad_batch(ds, batch_size, padding=padding, padded_shapes=pad_shape), # apply reduce funtion to pad
        window_size=window_size)


        
    return grouped_dataset

def shufflebatch(
        batch_size=256,
        shuffle_buffer=None,
        repeat=1,
        path_to_data="./data/SwissProt/sprot_ints.fasta"
):
    """
    Draws from an (optionally shuffled) dataset, repeats dataset repeat times,
    and serves batches of the specified size.
    """
    
    path_to_data = os.path.join(path_to_data)
    # Parse strings to tensors
    dataset = tf.contrib.data.TextLineDataset(path_to_data).map(tf_seq_to_tensor)
    if shuffle_buffer:
        # Stream elements uniformly randomly from a buffer
        dataset = dataset.shuffle(buffer_size=shuffle_buffer)
    # Apply a repeat. Because this is after the shuffle, all elements of the dataset should be seen before repeat.
    # See https://stackoverflow.com/questions/44132307/tf-contrib-data-dataset-repeat-with-shuffle-notice-epoch-end-mixed-epochs
    dataset = dataset.repeat(count=repeat)
    dataset = dataset.batch(batch_size)
    return dataset

In [0]:
#@title mLSTM_babbler
"""
The trained 1900-dimensional mLSTM babbler.
"""

import tensorflow as tf
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
import os

# Helpers
def tf_get_shape(tensor):
    static_shape = tensor.shape.as_list()
    dynamic_shape = tf.unstack(tf.shape(tensor))
    dims = [s[1] if s[0] is None else s[0]
            for s in zip(static_shape, dynamic_shape)]
    return dims

def sample_with_temp(logits, t):
    """
    Takes temperature between 0 and 1 -> zero most conservative, 1 most liberal. Samples.
    """
    t_adjusted = logits / t  # broadcast temperature normalization
    softed = tf.nn.softmax(t_adjusted)
    
    # Make a categorical distribution from the softmax and sample
    return tf.distributions.Categorical(probs=softed).sample()

def initialize_uninitialized(sess):
    """
    from https://stackoverflow.com/questions/35164529/in-tensorflow-is-there-any-way-to-just-initialize-uninitialised-variables
    """
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))


# Setup to initialize from the correctly named model files.
class mLSTMCell1900(tf.nn.rnn_cell.RNNCell):

    def __init__(self,
                 num_units,
                 model_path="./",
                 wn=True,
                 scope='mlstm',
                 var_device='cpu:0',
                 ):
        # Really not sure if I should reuse here
        super(mLSTMCell1900, self).__init__()
        self._num_units = num_units
        self._model_path = model_path
        self._wn = wn
        self._scope = scope
        self._var_device = var_device

    @property
    def state_size(self):
        # The state is a tuple of c and h
        return (self._num_units, self._num_units)

    @property
    def output_size(self):
        # The output is h
        return (self._num_units)

    def zero_state(self, batch_size, dtype):
        c = tf.zeros([batch_size, self._num_units], dtype=dtype)
        h = tf.zeros([batch_size, self._num_units], dtype=dtype)
        return (c, h)

    def call(self, inputs, state):
        # Inputs will be a [batch_size, input_dim] tensor.
        # Eg, input_dim for a 10-D embedding is 10
        nin = inputs.get_shape()[1].value

        # Unpack the state tuple
        c_prev, h_prev = state
        with tf.variable_scope(self._scope):
            wx_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_wx:0.npy"))
            wh_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_wh:0.npy"))
            wmx_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_wmx:0.npy"))
            wmh_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_wmh:0.npy"))
            b_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_b:0.npy"))
            gx_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_gx:0.npy"))
            gh_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_gh:0.npy"))
            gmx_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_gmx:0.npy"))
            gmh_init = np.load(os.path.join(self._model_path, "rnn_mlstm_mlstm_gmh:0.npy"))        
            wx = tf.get_variable(
                "wx", initializer=wx_init)
            wh = tf.get_variable(
                "wh", initializer=wh_init)
            wmx = tf.get_variable(
                "wmx", initializer=wmx_init)
            wmh = tf.get_variable(
                "wmh", initializer=wmh_init)
            b = tf.get_variable(
                "b", initializer=b_init)
            if self._wn:
                gx = tf.get_variable(
                    "gx", initializer=gx_init)
                gh = tf.get_variable(
                    "gh", initializer=gh_init)
                gmx = tf.get_variable(
                    "gmx", initializer=gmx_init)
                gmh = tf.get_variable(
                    "gmh", initializer=gmh_init)

        if self._wn:
            wx = tf.nn.l2_normalize(wx, dim=0) * gx
            wh = tf.nn.l2_normalize(wh, dim=0) * gh
            wmx = tf.nn.l2_normalize(wmx, dim=0) * gmx
            wmh = tf.nn.l2_normalize(wmh, dim=0) * gmh
        m = tf.matmul(inputs, wmx) * tf.matmul(h_prev, wmh)
        z = tf.matmul(inputs, wx) + tf.matmul(m, wh) + b
        i, f, o, u = tf.split(z, 4, 1)
        i = tf.nn.sigmoid(i)
        f = tf.nn.sigmoid(f)
        o = tf.nn.sigmoid(o)
        u = tf.tanh(u)
        c = f * c_prev + i * u
        h = o * tf.tanh(c)
        return h, (c, h)

class mLSTMCell(tf.nn.rnn_cell.RNNCell):

    def __init__(self,
                 num_units,
                 wx_init=tf.orthogonal_initializer(),
                 wh_init=tf.orthogonal_initializer(),
                 wmx_init=tf.orthogonal_initializer(),
                 wmh_init=tf.orthogonal_initializer(),
                 b_init=tf.orthogonal_initializer(),
                 gx_init=tf.ones_initializer(),
                 gh_init=tf.ones_initializer(),
                 gmx_init=tf.ones_initializer(),
                 gmh_init=tf.ones_initializer(),
                 wn=True,
                 scope='mlstm',
                 var_device='cpu:0',
                 ):
        # Really not sure if I should reuse here
        super(mLSTMCell, self).__init__()
        self._num_units = num_units
        self._wn = wn
        self._scope = scope
        self._var_device = var_device
        self._wx_init = wx_init
        self._wh_init = wh_init
        self._wmx_init = wmx_init
        self._wmh_init = wmh_init
        self._b_init = b_init
        self._gx_init = gx_init
        self._gh_init = gh_init
        self._gmx_init = gmx_init
        self._gmh_init = gmh_init

    @property
    def state_size(self):
        # The state is a tuple of c and h
        return (self._num_units, self._num_units)

    @property
    def output_size(self):
        # The output is h
        return (self._num_units)

    def zero_state(self, batch_size, dtype):
        c = tf.zeros([batch_size, self._num_units], dtype=dtype)
        h = tf.zeros([batch_size, self._num_units], dtype=dtype)
        return (c, h)

    def call(self, inputs, state):
        # Inputs will be a [batch_size, input_dim] tensor.
        # Eg, input_dim for a 10-D embedding is 10
        nin = inputs.get_shape()[1].value

        # Unpack the state tuple
        c_prev, h_prev = state
        with tf.variable_scope(self._scope):
            wx = tf.get_variable(
                "wx", initializer=self._wx_init)
            wh = tf.get_variable(
                "wh", initializer=self._wh_init)
            wmx = tf.get_variable(
                "wmx", initializer=self._wmx_init)
            wmh = tf.get_variable(
                "wmh", initializer=self._wmh_init)
            b = tf.get_variable(
                "b", initializer=self._b_init)
            if self._wn:
                gx = tf.get_variable(
                    "gx", initializer=self._gx_init)
                gh = tf.get_variable(
                    "gh", initializer=self._gh_init)
                gmx = tf.get_variable(
                    "gmx", initializer=self._gmx_init)
                gmh = tf.get_variable(
                    "gmh", initializer=self._gmh_init)

        if self._wn:
            wx = tf.nn.l2_normalize(wx, dim=0) * gx
            wh = tf.nn.l2_normalize(wh, dim=0) * gh
            wmx = tf.nn.l2_normalize(wmx, dim=0) * gmx
            wmh = tf.nn.l2_normalize(wmh, dim=0) * gmh
        m = tf.matmul(inputs, wmx) * tf.matmul(h_prev, wmh)
        z = tf.matmul(inputs, wx) + tf.matmul(m, wh) + b
        i, f, o, u = tf.split(z, 4, 1)
        i = tf.nn.sigmoid(i)
        f = tf.nn.sigmoid(f)
        o = tf.nn.sigmoid(o)
        u = tf.tanh(u)
        c = f * c_prev + i * u
        h = o * tf.tanh(c)
        return h, (c, h)

class mLSTMCellStackNPY(tf.nn.rnn_cell.RNNCell):

    def __init__(self,
                 num_units=256,
                 num_layers=4,
                 dropout=None,
                 res_connect=False,
                 wn=True,
                 scope='mlstm_stack',
                 var_device='cpu:0',
                 model_path="./"
                 ):
        # Really not sure if I should reuse here
        super(mLSTMCellStackNPY, self).__init__()
        self._model_path=model_path
        self._num_units = num_units
        self._num_layers = num_layers
        self._dropout = dropout
        self._res_connect = res_connect
        self._wn = wn
        self._scope = scope
        self._var_device = var_device
        bs = "rnn_mlstm_stack_mlstm_stack" # base scope see weight file names
        join = lambda x: os.path.join(self._model_path, x)
        layers = [mLSTMCell(
            num_units=self._num_units,
            wn=self._wn,
            scope=self._scope + str(i),
            var_device=self._var_device,
            wx_init=np.load(join(bs + "{0}_mlstm_stack{1}_wx:0.npy".format(i,i))),
            wh_init=np.load(join(bs + "{0}_mlstm_stack{1}_wh:0.npy".format(i,i))),
            wmx_init=np.load(join(bs + "{0}_mlstm_stack{1}_wmx:0.npy".format(i,i))),
            wmh_init=np.load(join(bs + "{0}_mlstm_stack{1}_wmh:0.npy".format(i,i))),
            b_init=np.load(join(bs + "{0}_mlstm_stack{1}_b:0.npy".format(i,i))),
            gx_init=np.load(join(bs + "{0}_mlstm_stack{1}_gx:0.npy".format(i,i))),
            gh_init=np.load(join(bs + "{0}_mlstm_stack{1}_gh:0.npy".format(i,i))),
            gmx_init=np.load(join(bs + "{0}_mlstm_stack{1}_gmx:0.npy".format(i,i))),
            gmh_init=np.load(join(bs + "{0}_mlstm_stack{1}_gmh:0.npy".format(i,i)))      
                 ) for i in range(self._num_layers)]
        if self._dropout:
            layers = [
                tf.contrib.rnn.DropoutWrapper(
                    layer, output_keep_prob=1-self._dropout) for layer in layers[:-1]] + layers[-1:]
        self._layers = layers

    @property
    def state_size(self):
        # The state is a tuple of c and h
        return (
            tuple(self._num_units for _ in range(self._num_layers)), 
            tuple(self._num_units for _ in range(self._num_layers))
            )

    @property
    def output_size(self):
        # The output is h
        return (self._num_units)

    def zero_state(self, batch_size, dtype):
        c_stack = tuple(tf.zeros([batch_size, self._num_units], dtype=dtype) for _ in range(self._num_layers))
        h_stack = tuple(tf.zeros([batch_size, self._num_units], dtype=dtype) for _ in range(self._num_layers))
        return (c_stack, h_stack)

    def call(self, inputs, state):
        # Inputs will be a [batch_size, input_dim] tensor.
        # Eg, input_dim for a 10-D embedding is 10

        # Unpack the state tuple
        c_prev, h_prev = state
        
        new_outputs = []
        new_cs = []
        new_hs = []
        for i, layer in enumerate(self._layers):
            if i == 0:
                h, (c,h_state) = layer(inputs, (c_prev[i],h_prev[i]))
            else:
                h, (c,h_state) = layer(new_outputs[-1], (c_prev[i],h_prev[i]))
            new_outputs.append(h)
            new_cs.append(c)
            new_hs.append(h_state)
        
        if self._res_connect:
            # Make sure number of layers does not affect the scale of the output
            scale_factor = tf.constant(1 / float(self._num_layers))
            final_output = tf.scalar_mul(scale_factor,tf.add_n(new_outputs))
        else:
            final_output = new_outputs[-1]

        return final_output, (tuple(new_cs), tuple(new_hs))

    
class babbler1900():

    def __init__(self,
                 model_path="./pbab_weights",
                 batch_size=256
                 ):
        self._rnn_size = 1900
        self._vocab_size = 26
        self._embed_dim = 10
        self._wn = True
        self._shuffle_buffer = 10000
        self._model_path = model_path
        self._batch_size = batch_size
        self._batch_size_placeholder = tf.placeholder(tf.int32, shape=[], name="batch_size")
        self._minibatch_x_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_x")
        self._initial_state_placeholder = (
            tf.placeholder(tf.float32, shape=[None, self._rnn_size]),
            tf.placeholder(tf.float32, shape=[None, self._rnn_size])
        )
        self._minibatch_y_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_y")
        # Batch size dimensional placeholder which gives the
        # Lengths of the input sequence batch. Used to index into
        # The final_hidden output and select the stop codon -1
        # final hidden for the graph operation.
        self._seq_length_placeholder = tf.placeholder(
            tf.int32, shape=[None], name="seq_len")
        self._temp_placeholder = tf.placeholder(tf.float32, shape=[], name="temp")
        rnn = mLSTMCell1900(self._rnn_size,
                    model_path=model_path,
                        wn=self._wn)
        zero_state = rnn.zero_state(self._batch_size, tf.float32)
        single_zero = rnn.zero_state(1, tf.float32)
        mask = tf.sign(self._minibatch_y_placeholder)  # 1 for nonpad, zero for pad
        inverse_mask = 1 - mask  # 0 for nonpad, 1 for pad

        total_padded = tf.reduce_sum(inverse_mask)

        pad_adjusted_targets = (self._minibatch_y_placeholder - 1) + inverse_mask

        embed_matrix = tf.get_variable(
            "embed_matrix", dtype=tf.float32, initializer=np.load(os.path.join(self._model_path, "embed_matrix:0.npy"))
        )
        embed_cell = tf.nn.embedding_lookup(embed_matrix, self._minibatch_x_placeholder)
        self._output, self._final_state = tf.nn.dynamic_rnn(
            rnn,
            embed_cell,
            initial_state=self._initial_state_placeholder,
            swap_memory=True,
            parallel_iterations=1
        )
        
        # If we are training a model on top of the rep model, we need to access
        # the final_hidden rep from output. Recall we are padding these sequences
        # to max length, so the -1 position will not necessarily be the right rep.
        # to get the right rep, I will use the provided sequence length to index.
        # Subtract one for the last place
        indices = self._seq_length_placeholder - 1
        self._top_final_hidden = tf.gather_nd(self._output, tf.stack([tf.range(tf_get_shape(self._output)[0], dtype=tf.int32), indices], axis=1))
        # LEFTOFF self._output is a batch size, seq_len, num_hidden.
        # I want to average along num_hidden, but I'll have to figure out how to mask out
        # the dimensions along sequence_length which are longer than the given sequence.
        flat = tf.reshape(self._output, [-1, self._rnn_size])
        logits_flat = tf.contrib.layers.fully_connected(
            flat, self._vocab_size - 1, activation_fn=None,
            weights_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_weights:0.npy"))),
            biases_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_biases:0.npy"))))
        self._logits = tf.reshape(
            logits_flat, [batch_size, tf_get_shape(self._minibatch_x_placeholder)[1], self._vocab_size - 1])
        batch_losses = tf.contrib.seq2seq.sequence_loss(
            self._logits,
            tf.cast(pad_adjusted_targets, tf.int32),
            tf.cast(mask, tf.float32),
            average_across_batch=False
        )
        self._loss = tf.reduce_mean(batch_losses)
        self._sample = sample_with_temp(self._logits, self._temp_placeholder)
        with tf.Session() as sess:
            self._zero_state = sess.run(zero_state)
            self._single_zero = sess.run(single_zero)

     
    def get_rep(self,seq):
        """
        Input a valid amino acid sequence, 
        outputs a tuple of average hidden, final hidden, final cell representation arrays.
        Unfortunately, this method accepts one sequence at a time and is as such quite
        slow.
        """
        with tf.Session() as sess:
            initialize_uninitialized(sess)
            # Strip any whitespace and convert to integers with the correct coding
            int_seq = aa_seq_to_int(seq.strip())[:-1]
            # Final state is a cell_state, hidden_state tuple. Output is
            # all hidden states
            final_state_, hs = sess.run(
                [self._final_state, self._output], feed_dict={
                    self._batch_size_placeholder: 1,
                    self._minibatch_x_placeholder: [int_seq],
                    self._initial_state_placeholder: self._zero_state}
            )

        final_cell, final_hidden = final_state_
        # Drop the batch dimension so it is just seq len by
        # representation size
        final_cell = final_cell[0]
        final_hidden = final_hidden[0]
        hs = hs[0]
        avg_hidden = np.mean(hs, axis=0)
        return avg_hidden, final_hidden, final_cell

    def get_babble(self, seed, length=250, temp=1):
        """
        Return a babble at temperature temp (on (0,1] with 1 being the noisiest)
        starting with seed and continuing to length length.
        Unfortunately, this method accepts one sequence at a time and is as such quite
        slow.
        """
        with tf.Session() as sess:
            initialize_uninitialized(sess)
            int_seed = aa_seq_to_int(seed.strip())[:-1]
        
            # No need for padding because this is a single element
            seed_samples, final_state_ = sess.run(
                [self._sample, self._final_state], 
                feed_dict={
                    self._minibatch_x_placeholder: [int_seed],
                    self._initial_state_placeholder: self._zero_state, 
                    self._batch_size_placeholder: 1,
                    self._temp_placeholder: temp
                }
            )
            # Just the actual character prediction
            pred_int = seed_samples[0, -1] + 1
            seed = seed + int_to_aa[pred_int]
        
            for i in range(length - len(seed)):
                pred_int, final_state_ = sess.run(
                    [self._sample, self._final_state], 
                    feed_dict={
                        self._minibatch_x_placeholder: [[pred_int]],
                        self._initial_state_placeholder: final_state_, 
                        self._batch_size_placeholder: 1,
                        self._temp_placeholder: temp
                    }
                )
                pred_int = pred_int[0, 0] + 1
                seed = seed + int_to_aa[pred_int]
        return seed        
        
    def get_rep_ops(self):
        """
        Return tensorflow operations for the final_hidden state and placeholder.
        POSTPONED: Implement avg. hidden
        """
        return self._top_final_hidden, self._minibatch_x_placeholder, self._batch_size_placeholder, self._seq_length_placeholder, self._initial_state_placeholder
        
    def get_babbler_ops(self):
        """
        Return tensorflow operations for 
        the logits, masked loss, minibatch_x placeholder, minibatch y placeholder, batch_size placeholder, initial_state placeholder
        Use if you plan on using babbler1900 as an initialization for another babbler, 
        eg for fine tuning the babbler to babble a differenct distribution.
        """
        return self._logits, self._loss, self._minibatch_x_placeholder, self._minibatch_y_placeholder, self._batch_size_placeholder, self._initial_state_placeholder

    def dump_weights(self,sess,dir_name="./1900_weights"):
        """
        Saves the weights of the model in dir_name in the format required 
        for loading in this module. Must be called within a tf.Session
        For which the weights are already initialized.
        """
        vs = tf.trainable_variables()
        for v in vs:
            name = v.name
            value = sess.run(v)
            print(name)
            print(value)
            np.save(os.path.join(dir_name,name.replace('/', '_') + ".npy"), np.array(value))
            


    def format_seq(self,seq,stop=False):
        """
        Takes an amino acid sequence, returns a list of integers in the codex of the babbler.
        Here, the default is to strip the stop symbol (stop=False) which would have 
        otherwise been added to the end of the sequence. If you are trying to generate
        a rep, do not include the stop. It is probably best to ignore the stop if you are
        co-tuning the babbler and a top model as well.
        """
        if stop:
            int_seq = aa_seq_to_int(seq.strip())
        else:
            int_seq = aa_seq_to_int(seq.strip())[:-1]
        return int_seq


    def bucket_batch_pad(self,filepath, upper=2000, lower=50, interval=10):
        """
        Read sequences from a filepath, batch them into buckets of similar lengths, and
        pad out to the longest sequence.
        Upper, lower and interval define how the buckets are created.
        Any sequence shorter than lower will be grouped together, as with any greater 
        than upper. Interval defines the "walls" of all the other buckets.
        WARNING: Define large intervals for small datasets because the default behavior
        is to repeat the same sequence to fill a batch. If there is only one sequence
        within a bucket, it will be repeated batch_size -1 times to fill the batch.
        """
        self._bucket_upper = upper
        self._bucket_lower = lower
        self._bucket_interval = interval
        self._bucket = [self._bucket_lower + (i * self._bucket_interval) for i in range(int(self._bucket_upper / self._bucket_interval))]
        self._bucket_batch =  bucketbatchpad(
                    batch_size=self._batch_size,
                    pad_shape=([None]),
                    window_size=self._batch_size,
                    bounds=self._bucket,
                    path_to_data=filepath,
                    shuffle_buffer=self._shuffle_buffer,
                    repeat=None
        ).make_one_shot_iterator().get_next()
        return self._bucket_batch

    def split_to_tuple(self, seq_batch):
        """
        NOTICE THAT BY DEFAULT THIS STRIPS THE LAST CHARACTER.
        IF USING IN COMBINATION WITH format_seq then set stop=True there.
        Return a list of batch, target tuples.
        The input (array-like) should
        look like 
        1. . . . . . . . sequence_length
        .
        .
        .
        batch_size
        """
        q = None
        num_steps = seq_batch.shape[1]
        # Minibatches should start at zero index and go to -1
        # Don't even try to get what is happenning here its a brainfuck and
        # probably inefficient
        xypairs = [
            (seq_batch[:, :-1][:, idx:idx + num_steps], seq_batch[:, idx + 1:idx + num_steps + 1]) for idx in np.arange(len(seq_batch[0]))[0:-1:num_steps]
        ]
        if q:
            for e in xypairs:
                q.put(e)
        else:
            return xypairs[0]

    def is_valid_seq(self, seq, max_len=2000):
        """
        True if seq is valid for the babbler, False otherwise.
        """
        l = len(seq)
        valid_aas = "MRHKDESTNQCUGPAVIFYWLO"
        if (l < max_len) and set(seq) <= set(valid_aas):
            return True
        else:
            return False

class babbler256(babbler1900):
    """
    Tested get_rep and get_rep_ops, assumed rest was unaffected by subclassing.
    """

    def __init__(self,
                 model_path="./256_weights/",
                 batch_size=256
                 ):
        self._rnn_size = 256
        self._vocab_size = 26
        self._embed_dim = 10
        self._num_layers = 4
        self._wn = True
        self._shuffle_buffer = 10000
        self._model_path = model_path
        self._batch_size = batch_size
        self._batch_size_placeholder = tf.placeholder(tf.int32, shape=[], name="batch_size")
        self._minibatch_x_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_x")
        self._initial_state_placeholder = (
                tuple(tf.placeholder(tf.float32, shape=[None, self._rnn_size]) for _ in range(self._num_layers)),
                tuple(tf.placeholder(tf.float32, shape=[None, self._rnn_size]) for _ in range(self._num_layers))
    )
        self._minibatch_y_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_y")
        # Batch size dimensional placeholder which gives the
        # Lengths of the input sequence batch. Used to index into
        # The final_hidden output and select the stop codon -1
        # final hidden for the graph operation.
        self._seq_length_placeholder = tf.placeholder(
            tf.int32, shape=[None], name="seq_len")
        self._temp_placeholder = tf.placeholder(tf.float32, shape=[], name="temp")
        rnn = mLSTMCellStackNPY(num_units=self._rnn_size,
                            num_layers=self._num_layers,
                            model_path=model_path,
                            wn=self._wn)
        zero_state = rnn.zero_state(self._batch_size, tf.float32)
        single_zero = rnn.zero_state(1, tf.float32)
        mask = tf.sign(self._minibatch_y_placeholder)  # 1 for nonpad, zero for pad
        inverse_mask = 1 - mask  # 0 for nonpad, 1 for pad

        total_padded = tf.reduce_sum(inverse_mask)

        pad_adjusted_targets = (self._minibatch_y_placeholder - 1) + inverse_mask

        embed_matrix = tf.get_variable(
            "embed_matrix", dtype=tf.float32, initializer=np.load(os.path.join(self._model_path, "embed_matrix:0.npy"))
        )
        embed_cell = tf.nn.embedding_lookup(embed_matrix, self._minibatch_x_placeholder)
        self._output, self._final_state = tf.nn.dynamic_rnn(
            rnn,
            embed_cell,
            initial_state=self._initial_state_placeholder,
            swap_memory=True,
            parallel_iterations=1
        )
        
        # If we are training a model on top of the rep model, we need to access
        # the final_hidden rep from output. Recall we are padding these sequences
        # to max length, so the -1 position will not necessarily be the right rep.
        # to get the right rep, I will use the provided sequence length to index.
        # Subtract one for the last place
        indices = self._seq_length_placeholder - 1
        self._top_final_hidden = tf.gather_nd(self._output, tf.stack([tf.range(tf_get_shape(self._output)[0], dtype=tf.int32), indices], axis=1))
        # LEFTOFF self._output is a batch size, seq_len, num_hidden.
        # I want to average along num_hidden, but I'll have to figure out how to mask out
        # the dimensions along sequence_length which are longer than the given sequence.
        flat = tf.reshape(self._output, [-1, self._rnn_size])
        logits_flat = tf.contrib.layers.fully_connected(
            flat, self._vocab_size - 1, activation_fn=None,
            weights_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_weights:0.npy"))),
            biases_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_biases:0.npy"))))
        self._logits = tf.reshape(
            logits_flat, [batch_size, tf_get_shape(self._minibatch_x_placeholder)[1], self._vocab_size - 1])
        batch_losses = tf.contrib.seq2seq.sequence_loss(
            self._logits,
            tf.cast(pad_adjusted_targets, tf.int32),
            tf.cast(mask, tf.float32),
            average_across_batch=False
        )
        self._loss = tf.reduce_mean(batch_losses)
        self._sample = sample_with_temp(self._logits, self._temp_placeholder)
        with tf.Session() as sess:
            self._zero_state = sess.run(zero_state)
            self._single_zero = sess.run(single_zero)

    def get_rep(self,seq):
        """
        get_rep needs to be minorly adjusted to accomadate the different state size of the 
        stack.
        Input a valid amino acid sequence, 
        outputs a tuple of average hidden, final hidden, final cell representation arrays.
        Unfortunately, this method accepts one sequence at a time and is as such quite
        slow.
        """
        with tf.Session() as sess:
            initialize_uninitialized(sess)
            # Strip any whitespace and convert to integers with the correct coding
            int_seq = aa_seq_to_int(seq.strip())[:-1]
            # Final state is a cell_state, hidden_state tuple. Output is
            # all hidden states
            final_state_, hs = sess.run(
                [self._final_state, self._output], feed_dict={
                    self._batch_size_placeholder: 1,
                    self._minibatch_x_placeholder: [int_seq],
                    self._initial_state_placeholder: self._zero_state}
            )

        final_cell, final_hidden = final_state_
        # Because this is a deep model, each of final hidden and final cell is tuple of num_layers
        final_cell = final_cell[-1]
        final_hidden = final_hidden[-1]
        hs = hs[0]
        avg_hidden = np.mean(hs, axis=0)
        return avg_hidden, final_hidden[0], final_cell[0]


class babbler64(babbler256):
    """
    Tested get_rep and dump weights. Assumed rest unaffected by subclassing.
    """

    def __init__(self,
                 model_path="./64_weights/",
                 batch_size=256
                 ):
        self._rnn_size = 64
        self._vocab_size = 26
        self._embed_dim = 10
        self._num_layers = 4
        self._wn = True
        self._shuffle_buffer = 10000
        self._model_path = model_path
        self._batch_size = batch_size
        self._batch_size_placeholder = tf.placeholder(tf.int32, shape=[], name="batch_size")
        self._minibatch_x_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_x")
        self._initial_state_placeholder = (
                tuple(tf.placeholder(tf.float32, shape=[None, self._rnn_size]) for _ in range(self._num_layers)),
                tuple(tf.placeholder(tf.float32, shape=[None, self._rnn_size]) for _ in range(self._num_layers))
    )
        self._minibatch_y_placeholder = tf.placeholder(
            tf.int32, shape=[None, None], name="minibatch_y")
        # Batch size dimensional placeholder which gives the
        # Lengths of the input sequence batch. Used to index into
        # The final_hidden output and select the stop codon -1
        # final hidden for the graph operation.
        self._seq_length_placeholder = tf.placeholder(
            tf.int32, shape=[None], name="seq_len")
        self._temp_placeholder = tf.placeholder(tf.float32, shape=[], name="temp")
        rnn = mLSTMCellStackNPY(num_units=self._rnn_size,
                            num_layers=self._num_layers,
                            model_path=model_path,
                            wn=self._wn)
        zero_state = rnn.zero_state(self._batch_size, tf.float32)
        single_zero = rnn.zero_state(1, tf.float32)
        mask = tf.sign(self._minibatch_y_placeholder)  # 1 for nonpad, zero for pad
        inverse_mask = 1 - mask  # 0 for nonpad, 1 for pad

        total_padded = tf.reduce_sum(inverse_mask)

        pad_adjusted_targets = (self._minibatch_y_placeholder - 1) + inverse_mask

        embed_matrix = tf.get_variable(
            "embed_matrix", dtype=tf.float32, initializer=np.load(os.path.join(self._model_path, "embed_matrix:0.npy"))
        )
        embed_cell = tf.nn.embedding_lookup(embed_matrix, self._minibatch_x_placeholder)
        self._output, self._final_state = tf.nn.dynamic_rnn(
            rnn,
            embed_cell,
            initial_state=self._initial_state_placeholder,
            swap_memory=True,
            parallel_iterations=1
        )
        
        # If we are training a model on top of the rep model, we need to access
        # the final_hidden rep from output. Recall we are padding these sequences
        # to max length, so the -1 position will not necessarily be the right rep.
        # to get the right rep, I will use the provided sequence length to index.
        # Subtract one for the last place
        indices = self._seq_length_placeholder - 1
        self._top_final_hidden = tf.gather_nd(self._output, tf.stack([tf.range(tf_get_shape(self._output)[0], dtype=tf.int32), indices], axis=1))
        # LEFTOFF self._output is a batch size, seq_len, num_hidden.
        # I want to average along num_hidden, but I'll have to figure out how to mask out
        # the dimensions along sequence_length which are longer than the given sequence.
        flat = tf.reshape(self._output, [-1, self._rnn_size])
        logits_flat = tf.contrib.layers.fully_connected(
            flat, self._vocab_size - 1, activation_fn=None,
            weights_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_weights:0.npy"))),
            biases_initializer=tf.constant_initializer(np.load(os.path.join(self._model_path, "fully_connected_biases:0.npy"))))
        self._logits = tf.reshape(
            logits_flat, [batch_size, tf_get_shape(self._minibatch_x_placeholder)[1], self._vocab_size - 1])
        batch_losses = tf.contrib.seq2seq.sequence_loss(
            self._logits,
            tf.cast(pad_adjusted_targets, tf.int32),
            tf.cast(mask, tf.float32),
            average_across_batch=False
        )
        self._loss = tf.reduce_mean(batch_losses)
        self._sample = sample_with_temp(self._logits, self._temp_placeholder)
        with tf.Session() as sess:
            self._zero_state = sess.run(zero_state)
            self._single_zero = sess.run(single_zero)

In [0]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    # from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [0]:
batch_size = 12
b = babbler64(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

In [0]:
homstrad_seq = "MGDCANANVYPNWVSKDWAGGQPTHNEAGQSIVYKGNLYTANWYTASVPGSDSSWTQVGSCNPIMTAPAYVPGTTYAQGALVSYQGYVWQTKWGYITSAPGSDSAWLKVGRVAWQVNTAYTAGQLVTYNGKTYKCLQPHTSLAGWEPSNVPALWQL"

In [0]:
np.array(b.format_seq(homstrad_seq))


array([24,  1, 13,  5, 11, 15,  9, 15,  9, 16, 19, 14,  9, 20, 16,  7,  4,
        5, 20, 15, 13, 13, 10, 14,  8,  3,  9,  6, 15, 13, 10,  7, 17, 16,
       19,  4, 13,  9, 21, 19,  8, 15,  9, 20, 19,  8, 15,  7, 16, 14, 13,
        7,  5,  7,  7, 20,  8, 10, 16, 13,  7, 11,  9, 14, 17,  1,  8, 15,
       14, 15, 19, 16, 14, 13,  8,  8, 19, 15, 10, 13, 15, 21, 16,  7, 19,
       10, 13, 19, 16, 20, 10,  8,  4, 20, 13, 19, 17,  8,  7, 15, 14, 13,
        7,  5,  7, 15, 20, 21,  4, 16, 13,  2, 16, 15, 20, 10, 16,  9,  8,
       15, 19,  8, 15, 13, 10, 21, 16,  8, 19,  9, 13,  4,  8, 19,  4, 11,
       21, 10, 14,  3,  8,  7, 21, 15, 13, 20,  6, 14,  7,  9, 16, 14, 15,
       21, 20, 10, 21])

In [0]:
b.is_valid_seq(homstrad_seq)


True

In [0]:
avg, final, cell = b.get_rep(homstrad_seq)

In [0]:
avg

array([ 0.03034502,  0.13352253, -0.12410649, -0.92995155, -0.01301586,
       -0.46326855,  0.09718941,  0.21206318,  0.32108128, -0.44596186,
        0.03051127,  0.27033928,  0.03058002, -0.43970174, -0.06244392,
       -0.05468851, -0.9099609 ,  0.0325312 ,  0.18408118, -0.8002021 ,
       -0.09938771,  0.37391418, -0.5020082 , -0.2410639 ,  0.17328246,
       -0.06923638, -0.08664186,  0.01863937,  0.6368234 , -0.5976544 ,
        0.21782273,  0.32438406,  0.42798293, -0.34884828,  0.6454408 ,
        0.10704129, -0.2489908 ,  0.03792899,  0.11980145, -0.1857546 ,
        0.0052999 ,  0.39345473,  0.04663284, -0.01507659, -0.06532604,
        0.3527997 , -0.8653756 , -0.2939967 ,  0.22158687,  0.3030935 ,
        0.2957644 ,  0.04372757, -0.0396818 , -0.02977492,  0.13351855,
        0.03641961, -0.02092759,  0.06022185, -0.02610497,  0.26853037,
       -0.18686132, -0.06438694,  0.5965727 , -0.08861247], dtype=float32)

In [0]:
 def get_str_from_embedding(seq, embedding):
  for i in embedding:
    seq += " " + str(i) + " "
  seq = seq[:-1]
  return seq

In [0]:
import shutil
import fnmatch
shutil.unpack_archive("/content/drive/My Drive/Spring 2020/4761 Comp. Genomics/solutions.zip", "/content/Input")

In [0]:
!rm -rf '/content/Input'

In [0]:
family_to_num_seqs = {}
fams = os.listdir("/content/Input/solutions/")
fams.remove(".DS_Store")
path = "/content/Input/solutions/"
for family in fams:
    family_to_num_seqs[family] = len(fnmatch.filter(os.listdir(path+family), '*.txt'))
print(len(family_to_num_seqs))
family_to_num_seqs_filtered = {key:val for key, val in family_to_num_seqs.items() if val >= 5}
print(len(family_to_num_seqs_filtered))

1028
149


In [0]:
import os
base_dir = "/content/Input/solutions"

solutions_dir = os.listdir(base_dir)
first_itr = True
i = 0
with open("embeddings.txt", "w") as f2:
  for family in solutions_dir:
    i += 1
    print(i+ ". Fam: ", family)
    if family in family_to_num_seqs_filtered.keys():
      files = os.listdir(base_dir+"/" + family)
      for sequence_file in files:
        with open(base_dir+"/"+family+"/"+sequence_file, 'r') as f:
          seq = f.read()
          seq = seq.rstrip()
          avg_hidden_rep = b.get_rep(seq)[0]
          current_len = len(avg_hidden_rep)
          if not first_itr and current_len != previous_len:
            print("Mismatch")
          first_itr = False
          previous_len = current_len
          embedding = get_str_from_embedding(seq, avg_hidden_rep)
          f2.write(embedding)
          f2.write("\n")
        f.close()
  f2.close()

1
Fam:  Cereal trypsin_alpha-amylase inhibitor family
2
Fam:  Tobacco_mosaic_virus_coat
3
Fam:  Scorpion_short_toxin
4
Fam:  Lipocalin_family
5
Fam:  pancreatic_ribonuclease
6
Fam:  Fumarylacetoacetate_(FAA)_hydrolase_family
7
Fam:  amino_acid_dehydrogenase
8
Fam:  Cytosol_aminopeptidase_family,_N-terminal_domain
9
Fam:  Ribonuclease_U2
10
Fam:  avidin
11
Fam:  Cytochrome_b
12
Fam:  Ribosomal_protein_S5,_C-terminal_domain
13
Fam:  Carbon-nitrogen_hydrolase
14
Fam:  Ferrochelatase_
15
Fam:  Glutamate_synthase_central_domain
16
Fam:  cytochrome_c'
17
Fam:  Insulinase_(Peptidase_family_M16)
18
Fam:  tRNA_synthetases_class_I_(R),_catalytic_domain
19
Fam:  Nitrogen_regulatory_protein_P-II
20
Fam:  carbohydrate_binding_module_family_12
21
Fam:  Adenylosuccinate_synthetase
22
Fam:  _Low_molecular_weight_phosphatase
23
Fam:  alpha_beta-hydrolase
24
Fam:  Bacterial_regulatory_proteins,_luxR_family
25
Fam:  Pou_domain_-_N-terminal_to_homeobox_domain
26
Fam:  Methylaspartate_ammonia-lyase,_C-term

In [0]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=(10, 10))
ax1 = plt.axes()
x_axis = ax1.axes.get_xaxis()
x_axis.set_visible(False)
x = family_to_num_seqs.keys()
y = family_to_num_seqs.values()
plt.bar(x, y)
plt.title("Family Sizes", size=18)
plt.ylabel("No. of sequences", size = 16)
plt.show()

In [0]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=(10, 10))
ax1 = plt.axes()
x_axis = ax1.axes.get_xaxis()
x_axis.set_visible(False)
x = family_to_num_seqs_filtered.keys()
y = family_to_num_seqs_filtered.values()
plt.bar(x, y)
plt.title("Family Sizes", size=18)
plt.ylabel("No. of sequences", size = 16)
plt.show()